In [1]:
import httpx
import asyncio
import pandas as pd
import azure.storage.queue as asq
import os
import json
import time
from pathlib import Path

# variables used throughout
tciabase = 'https://services.cancerimagingarchive.net/services/v3/TCIA/query'
series_df_folder = 'c:\\githealth\\notebooks\\data\\'

In [101]:
# Aggregate all CSV files into a single dataframe
files = [x for x in Path(series_df_folder).glob('*.csv')] 
len(files)

97

In [102]:
df_series = pd.DataFrame()
for file in files:
    df = pd.read_csv(file)
    df_series = df_series.append(df, ignore_index=True, verify_integrity=True)
df_series = df_series.drop(df_series.columns[0], axis=1) # During merge pandas adds a munges the index so no data is potentially lost, drop it
df.head(2)

,Unnamed: 0,Collection,PatientID,PatientName,PatientSex,StudyInstanceUID,StudyDate,StudyDescription,PatientAge,SeriesCount,...,ProtocolName,SeriesDate,SeriesDescription,BodyPartExamined,SeriesNumber,Manufacturer,ManufacturerModelName,SoftwareVersions,Visibility,ImageCount
0,0,TCGA-UCEC,TCGA-D1-A169,TCGA-D1-A169,F,1.3.6.1.4.1.14519.5.2.1.3344.4020.247048849237...,1995-10-01,CT ABDOMEN w + PELVIS w,069Y,3,...,02_0_Abdomen_Pelvis_Oncology,1995-10-01,Topogram 0.6 T20s,UTERUS,2.0,SIEMENS,Sensation 64,syngo CT 2009E,1,2
1,1,TCGA-UCEC,TCGA-D1-A169,TCGA-D1-A169,F,1.3.6.1.4.1.14519.5.2.1.3344.4020.247048849237...,1995-10-01,CT ABDOMEN w + PELVIS w,069Y,3,...,02_0_Abdomen_Pelvis_Oncology,1995-10-01,AP 5.0 B30f,UTERUS,3.0,SIEMENS,Sensation 64,syngo CT 2009E,1,97


In [103]:
print(len(df_series))
df_series.head()

120888


,Collection,PatientID,PatientName,PatientSex,StudyInstanceUID,StudyDate,StudyDescription,PatientAge,SeriesCount,SeriesInstanceUID,...,BodyPartExamined,SeriesNumber,Manufacturer,ManufacturerModelName,SoftwareVersions,Visibility,ImageCount,Unnamed: 0.1,Unnamed: 0.1.1,AnnotationsFlag
0,4D-Lung,100_HM10395,P100,M,1.3.6.1.4.1.14519.5.2.1.6834.5010.119076582718...,1997-10-28,p4,000Y,10.0,1.3.6.1.4.1.14519.5.2.1.6834.5010.790522551686...,...,LUNG,507.0,"Varian Imaging Laboratories, Switzerland",Trilogy Cone Beam CT,LightSpeedverrel,1.0,50.0,NaN,NaN,NaN
1,4D-Lung,100_HM10395,P100,M,1.3.6.1.4.1.14519.5.2.1.6834.5010.119076582718...,1997-10-28,p4,000Y,10.0,1.3.6.1.4.1.14519.5.2.1.6834.5010.247324515019...,...,LUNG,508.0,"Varian Imaging Laboratories, Switzerland",Trilogy Cone Beam CT,LightSpeedverrel,1.0,50.0,NaN,NaN,NaN
2,4D-Lung,100_HM10395,P100,M,1.3.6.1.4.1.14519.5.2.1.6834.5010.119076582718...,1997-10-28,p4,000Y,10.0,1.3.6.1.4.1.14519.5.2.1.6834.5010.288280716330...,...,LUNG,505.0,"Varian Imaging Laboratories, Switzerland",Trilogy Cone Beam CT,LightSpeedverrel,1.0,50.0,NaN,NaN,NaN
3,4D-Lung,100_HM10395,P100,M,1.3.6.1.4.1.14519.5.2.1.6834.5010.119076582718...,1997-10-28,p4,000Y,10.0,1.3.6.1.4.1.14519.5.2.1.6834.5010.336704149861...,...,LUNG,503.0,"Varian Imaging Laboratories, Switzerland",Trilogy Cone Beam CT,LightSpeedverrel,1.0,50.0,NaN,NaN,NaN
4,4D-Lung,100_HM10395,P100,M,1.3.6.1.4.1.14519.5.2.1.6834.5010.119076582718...,1997-10-28,p4,000Y,10.0,1.3.6.1.4.1.14519.5.2.1.6834.5010.220026873632...,...,LUNG,504.0,"Varian Imaging Laboratories, Switzerland",Trilogy Cone Beam CT,LightSpeedverrel,1.0,50.0,NaN,NaN,NaN


In [120]:
df = df_series.describe(include='all').head(2)
#df.to_csv('C:\\githealth\\data\\TciaSummaryStats.csv')
df

,Collection,PatientID,PatientName,PatientSex,StudyInstanceUID,StudyDate,StudyDescription,PatientAge,SeriesCount,SeriesInstanceUID,...,BodyPartExamined,SeriesNumber,Manufacturer,ManufacturerModelName,SoftwareVersions,Visibility,ImageCount,Unnamed: 0.1,Unnamed: 0.1.1,AnnotationsFlag
count,120888,120888,117280,110731,120888,120888,111126,95439,120888.0,120888,...,93911,119002.0,114299,113448,109431,120888.0,120888.0,1398.0,1398.0,1201.0
unique,96,10264,8984,5,18915,6754,3155,118,NaN,120888,...,74,NaN,84,289,511,NaN,NaN,NaN,NaN,NaN


In [105]:
df_series.columns

Index(['Collection', 'PatientID', 'PatientName', 'PatientSex',
       'StudyInstanceUID', 'StudyDate', 'StudyDescription', 'PatientAge',
       'SeriesCount', 'SeriesInstanceUID', 'Modality', 'ProtocolName',
       'SeriesDate', 'SeriesDescription', 'BodyPartExamined', 'SeriesNumber',
       'Manufacturer', 'ManufacturerModelName', 'SoftwareVersions',
       'Visibility', 'ImageCount', 'Unnamed: 0.1', 'Unnamed: 0.1.1',
       'AnnotationsFlag'],
      dtype='object')

In [106]:
# Unique values so far
# 96 collections
# 

#df_series['SeriesCount'].head(25)
df_trimmed = df_series[['Collection', 'PatientID', 'StudyInstanceUID', 
                        'SeriesCount', 'SeriesInstanceUID', 'SeriesNumber', 'ImageCount']]


In [107]:
df_trimmed['ImageCount'].sum()

26118205.0

In [2]:
#df_series.to_csv('C:\\githealth\\data\\TciaSeriesDataForAllCollections.csv')


In [4]:
df = pd.read_csv('C:\\githealth\\notebooks\\data\\TciaSeriesDataForAllCollections.csv')

In [5]:
len(df)

120888

In [7]:
df.columns


Index(['Unnamed: 0', 'Collection', 'PatientID', 'PatientName', 'PatientSex',
       'StudyInstanceUID', 'StudyDate', 'StudyDescription', 'PatientAge',
       'SeriesCount', 'SeriesInstanceUID', 'Modality', 'ProtocolName',
       'SeriesDate', 'SeriesDescription', 'BodyPartExamined', 'SeriesNumber',
       'Manufacturer', 'ManufacturerModelName', 'SoftwareVersions',
       'Visibility', 'ImageCount', 'Unnamed: 0.1', 'Unnamed: 0.1.1',
       'AnnotationsFlag'],
      dtype='object')

In [14]:
df2 = df[['Collection', 'StudyInstanceUID', 'SeriesCount', 'SeriesInstanceUID', 'ImageCount']]

In [15]:
len(df2)

120888

In [17]:
df2.tail()

,Collection,StudyInstanceUID,SeriesCount,SeriesInstanceUID,ImageCount
120883,TCGA-UCEC,1.3.6.1.4.1.14519.5.2.1.1706.4020.167204261356...,3.0,1.3.6.1.4.1.14519.5.2.1.1706.4020.572542437976...,106.0
120884,TCGA-UCEC,1.3.6.1.4.1.14519.5.2.1.1706.4020.103744621346...,4.0,1.3.6.1.4.1.14519.5.2.1.1706.4020.984163594324...,39.0
120885,TCGA-UCEC,1.3.6.1.4.1.14519.5.2.1.1706.4020.103744621346...,4.0,1.3.6.1.4.1.14519.5.2.1.1706.4020.259558002268...,143.0
120886,TCGA-UCEC,1.3.6.1.4.1.14519.5.2.1.1706.4020.103744621346...,4.0,1.3.6.1.4.1.14519.5.2.1.1706.4020.282820206564...,282.0
120887,TCGA-UCEC,1.3.6.1.4.1.14519.5.2.1.1706.4020.103744621346...,4.0,1.3.6.1.4.1.14519.5.2.1.1706.4020.282854123295...,2.0


In [19]:
len(df.groupby('Collection'))

96

In [20]:
groups = df.groupby('Collection')['ImageCount'].sum()

In [29]:
groups.sort_values()

Collection
Lung Phantom                     345.0
Prostate-3T                     1258.0
TCGA-READ                       1796.0
PDMR-BL0293-F563                2222.0
RIDER PHANTOM PET-CT            2231.0
                               ...    
ACRIN-FLT-Breast              688598.0
IvyGAP                        846743.0
CT COLONOGRAPHY               941771.0
Phantom FDA                  1468751.0
LDCT-and-Projection-data    13027273.0
Name: ImageCount, Length: 96, dtype: float64

In [35]:
---------------------------------------------------

In [36]:
len(x)

480

## 